## Import Libraries

In [40]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import time
import glob


# useful helper function
from helpers import imshow

## Helper Methods

In [2]:
def extractFeaturesFromImage(query_path):
    '''
    @query_path: path of the query image
    @returns: keypoints and descriptors of the query image
    '''
    query_img = cv2.imread(query_path, 0)
    kp_query, des_query = sift.detectAndCompute(query_img, None)  
    return kp_query, des_query, query_img

In [3]:
def initializeMatcher():
    '''
    @returns: FLANN matcher
    '''
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    return flann

In [4]:
def initializeCamera(w, camera_port):
    '''
    @w: width of the video frame
    @returns: camera object
    '''
    camera = cv2.VideoCapture(camera_port)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
    camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
    return camera

In [101]:
def getGoodMatches(des_query, des_scene):
    '''
    @des_query: descriptors of a query image
    @des_scene: descriptors of a scene image
    @returns: list of good matches for query and scene images
    '''
    
    matches = flann.knnMatch(des_query, des_scene,k=2)

    # ratio test as per Lowe's paper
    good_matches = []
    
    # Each member of the matches list must be checked whether two neighbours really exist.
    for m_n in matches:
        if len(m_n) != 2:
            continue
        (m,n) = m_n
        if m.distance < 0.7*n.distance:
            good_matches.append(m)
            
    return good_matches

In [43]:
def importSpines(directory = 'spines'):
    '''
    @directory: folder where spine images are located
    @returns: array of all grayscale spine images represented as np.arrays, 
    their keypoints and descriptors
    '''
    directory += '/*.jpg'
    spine_fnames = glob.glob(directory)    
    spines = []
    
    for fname in spine_fnames:
        spines.append(extractFeaturesFromImage(fname))
        
    return spines

In [120]:
def importARWindows(spines, dir_covers = 'covers', dir_info = 'info'):
    '''
    @dir_covers: folder where book covers are located
    @dir_info: folder where book info windows are located
    @returns: array of cover window and info window for each book. 
    '''
    dir_covers += '/*.jpg'
    dir_info += '/*.jpg'
    
    cover_fnames = glob.glob(dir_covers)    
    info_fnames = glob.glob(dir_info)    
    
    if len(cover_fnames) != len(info_fnames):
        print "Number of cover windows and info windows is different"
        return None
    
    ar_windows = []

    for i in range(len(cover_fnames)):
        target_height = spines[i][2].shape[0]  # height of the book spine image       
        
        # Cover window
        cover_fname = cover_fnames[i]
        cover_image = cv2.imread(cover_fname)
        # Fit image to book spine
        scaling_factor_cover = 1.0*target_height/cover_image.shape[0]
        cover_image = cv2.resize(cover_image, None, fx = scaling_factor_cover, fy = scaling_factor_cover, interpolation = cv2.INTER_AREA)
        
        
        # Book info window
        info_fname = info_fnames[i]
        info_image = cv2.imread(info_fname)
        # Fit image to book spine
        scaling_factor_info = 1.0*target_height/info_image.shape[0]
        info_image = cv2.resize(info_image, None, fx = scaling_factor_info, fy = scaling_factor_info, interpolation = cv2.INTER_AREA)
        
        ar_windows.append([cover_image, info_image])
    return ar_windows


In [121]:
def getGoodMatchesForAllSpines(spines, des_scene):
    '''
    @spine: array containing spine keypoints, descriptors and images
    @descriptors of the scene image (video frame)
    @returns: good matches for each of the book spines
    '''
    good_matches = []
    
    for spine in spines:
        des_query = spine[1]
        good_matches.append(getGoodMatches(des_query,des_scene))
    return good_matches

In [129]:
def isSelectedInThisFrame(good_matches, query_img, frame, kp_query, kp_scene):
    '''
    @good_matches: set of good matches
    @query_img: query image
    @frame: video frame image
    @kp_query: keypoints of the query image
    @kp_scene: keypoints of the scene image
    @returns: True if the book was selected in this frame, and False otherwise
    
    ASSUMPTION: keypoints on the book spine are distributed uniformly across the height
    '''
    if len(good_matches) > 15:
        
        # Source points and destnation points
        src_pts = np.float32([kp_query[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # Compute Homography
        M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

        rows,cols = query_img.shape[:2]
        dst = cv2.warpPerspective(frame, M, (cols, rows));
        
        # Theshold matched points that are below the book spine.
        # If less than 1/5 of all good matches i from the lower part of the book spine,
        # register selection of the book spine.
        threshold_height = int(query_img.shape[0] * 0.7)        
        num_matches_in_lower_part = 0
        for point in src_pts:
            if point[0][1] >= threshold_height:
                   num_matches_in_lower_part += 1
        
        book_selected = False
        threshold = int(len(good_matches) / 10)
        print "BOOKSPINE is found"
        print "   ", threshold, num_matches_in_lower_part        
        selected_status = True if num_matches_in_lower_part <= threshold else False
        return selected_status
    
    return False

In [130]:
def checkSelected(good_matches, spines, frame, kp_scene):
    '''
    @good_matches: array of good matches for each book spine
    @spines: keypoints, descriptors and images for all spines
    @frame: current frame
    @kp_scene: keypoints of the current scene (video frame)
    @returns: book that was selected in the current scene, and -1 if no book
    was selected
    '''
    book_selected_in_this_frame = -1
    for i in range(len(spines)):
        query_img = spines[i][2]
        kp_query = spines[i][0]
        if isSelectedInThisFrame(good_matches[i], query_img, frame, kp_query, kp_scene):
            book_selected_in_this_frame = i
            break
    return book_selected_in_this_frame

In [135]:
def updateSelectedStatus(book_selected_in_this_frame, book_selected_in_prev_frame, num_frames_selected):
    '''
    @book_selected_in_this_frame: book selected in the current frame
    @book_selected_in_prev_frame: book selected in the previous frame
    @num_frames_selected: number of consecutive frames that the book was selected
    @returns augmented_book_id: ID of the augmented book, and -1 if no book is 
    selected to be augmented
    @returns num_frames_selected: updated number of consecutive frames that the book 
    was selected
    '''
    
    print "book_selected_in_this_frame", book_selected_in_this_frame
    print "book_selected_in_prev_frame", book_selected_in_prev_frame
    print "NUM FRAMES SELECTED", num_frames_selected
    if book_selected_in_this_frame == -1:
        num_frames_selected = 0
    elif book_selected_in_this_frame == book_selected_in_prev_frame:
        num_frames_selected += 1
    else:
        num_frames_selected = 1

    if num_frames_selected >= 10:
        num_frames_selected = 0
        augmented_book_id = book_selected_in_this_frame
    else:    
        augmented_book_id = -1
    
    return augmented_book_id, num_frames_selected


In [155]:
def augmentSpine(good_matches, kp_scene, kp_query, frame, window_id, arWindowsSpine):
    '''
    @good_matches: set of good matches for this spine
    @kp_scene: keypoints of the scene image (video frame)
    @kp_query: keypoints of the query image (spine)
    @frame: current video frame
    @window_id: id of the spine's window with which the spine will be augmented
    @arWindowsSpine: libary of augmenting windows foe this spine
    @returns: image of the augmented video frame
    
    '''
    if len(good_matches) > 15:
        # Image of the AR window with which the spine will be augmented
        AR_image = arWindowsSpine[window_id]

        # Source (spine) points and destnation (frame) points
        src_pts = np.float32([kp_query[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # Compute Homography
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        rows,cols = frame.shape[:2]
        dst = cv2.warpPerspective(AR_image, M, (cols, rows));
        
        frame_augmented = frame.copy()
        frame_augmented[np.nonzero(dst)] = dst[np.nonzero(dst)]
        return frame_augmented
    return frame

# Main system code

In [158]:
from IPython.display import clear_output

# SIFT keypoint extractor
sift = cv2.xfeatures2d.SIFT_create()

spines = importSpines()
arWindows = importARWindows(spines)

# Initialize FLANN matcher
flann = initializeMatcher()

# Initialize camera
camera = initializeCamera(640, 1)

book_selected_in_prev_frame = -1 # book selected in previous frame
num_frames_selected = 0 # number of consecutive frames in which the current
                        # book spine was selected
augmented_book_id = -1 # Book for which AR menu is displayed

aug_window_id = 0 # Initialize id of the augmenting window

while True:
    clear_output(True)

    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # keypoints and descriptors for video frame 
    kp_scene, des_scene =  sift.detectAndCompute(frame_gray,None)
    
    if augmented_book_id == -1:
        
        # Find good matches for each book spine
        good_matches = getGoodMatchesForAllSpines(spines, des_scene)
  
        # Check if any of the bookspines has been selected
        # If selected 10 times in a row, change this book's status to Selected
        book_selected_in_this_frame = checkSelected(good_matches, spines, frame, kp_scene)
        augmented_book_id, num_frames_selected = updateSelectedStatus(book_selected_in_this_frame, book_selected_in_prev_frame, num_frames_selected)
        
        book_selected_in_prev_frame = book_selected_in_this_frame
        
    else:
        print "SELECTED", augmented_book_id
        
        spine = spines[augmented_book_id]
        kp_query = spine[0]
        arWindowsSpine = arWindows[augmented_book_id]
        
        good_matches_aug = getGoodMatchesForAllSpines([spine], des_scene)[0]
        augmented_frame = augmentSpine(good_matches_aug, kp_scene, kp_query, frame, aug_window_id, arWindowsSpine)
        augmented_frame = cv2.flip(augmented_frame, 1)
        cv2.imshow("Augmented Frame", augmented_frame)
    
    if cv2.waitKey(5) == 32:
        augmented_book_id = -1
    elif cv2.waitKey(5) == 27:
        break  
   
    frame = cv2.flip(frame, 1)
    cv2.imshow("Frame", frame)
    
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

SELECTED 1
[[[  9   5   3]
  [ 10   6   4]
  [  8   6   4]
  ...
  [132 163 167]
  [128 165 165]
  [130 166 167]]

 [[ 11   6   7]
  [ 11   6   7]
  [  9   7   5]
  ...
  [136 162 171]
  [134 163 170]
  [132 162 169]]

 [[  7   8   5]
  [  7   8   5]
  [ 10   6   4]
  ...
  [137 159 169]
  [140 157 179]
  [139 156 178]]

 ...

 [[ 19  44  24]
  [ 19  44  24]
  [ 30  38  27]
  ...
  [112 119 132]
  [124 115 135]
  [125 117 136]]

 [[ 30  40  23]
  [ 29  39  21]
  [ 24  36  22]
  ...
  [122 118 130]
  [122 115 137]
  [122 115 137]]

 [[ 41  41  28]
  [ 34  34  21]
  [  7  22  13]
  ...
  [123 120 130]
  [122 115 137]
  [118 110 132]]]


-1

# IMPORT EVERYTHING

In [102]:
des_query = spines[0][1]

print getGoodMatches(des_query,des_scene)

[<DMatch 0x11d178210>]


In [98]:
print len(spines[0])

3


## Real time spine tracking and augmentation

In [57]:


query_path = 'book2.jpg'
query = cv2.imread(query_path, 0)



target_height = query.shape[0]



scaling_factor_q = 1.0*target_height/query.shape[0]
query = cv2.resize(query, None, fx = scaling_factor_q, fy = scaling_factor_q, interpolation = cv2.INTER_AREA)
print query.shape


AR_image_path = 'cover4.jpg'
AR_image = cv2.imread(AR_image_path)
scaling_factor_ar = 1.0*target_height/AR_image.shape[0]
AR_image = cv2.resize(AR_image, None, fx = scaling_factor_ar, fy = scaling_factor_ar, interpolation = cv2.INTER_AREA)
print AR_image.shape


# SIFT keypoint detector
sift = cv2.xfeatures2d.SIFT_create()

# Import the query image and compute SIFT keypoints and descriptors
kp_query, des_query = sift.detectAndCompute(query, None)  

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)

# Camera settings
camera = cv2.VideoCapture(0)
# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
camera.set(cv2.CAP_PROP_EXPOSURE,-4) 

while True:
    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # keypoints and descriptors for video frame 
    
    kp_scene, des_scene =  sift.detectAndCompute(gray,None)
       
    # Match scene descriptors with query descriptors
    matches = flann.knnMatch(des_query,des_scene,k=2)

    # ratio test as per Lowe's paper
    good_matches = []
    # Each member of the matches list must be checked whether two neighbours really exist.
    for m_n in matches:
        if len(m_n) != 2:
            continue
        (m,n) = m_n
        if m.distance < 0.7*n.distance:
            good_matches.append(m)
    
#     print "{} matches found".format(len(good_matches))

    # If enough matches found ...
    if len(good_matches) > 15:
        
        # Source points and destnation points
        src_pts = np.float32([kp_query[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        
        # Compute Homography
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        rows,cols = frame.shape[:2]
        dst = cv2.warpPerspective(AR_image, M, (cols, rows));

        frame[np.nonzero(dst)] = dst[np.nonzero(dst)]
    
        
#         # Draw a rectangle that marks the found model in the frame
#         h, w = query.shape
#         pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
#         # project corners into frame
#         dst = cv2.perspectiveTransform(pts, M)
    
#         # Draw lines
#         cv2.polylines(frame, [np.int32(dst)], True, (0, 255, 255), 3, cv2.LINE_AA) 
        
    
    if cv2.waitKey(5) == 32:
        imshow(frame)        
    elif cv2.waitKey(5) == 27:
        break  
   
    #frame = cv2.flip(frame, 1)
    cv2.imshow("SIFT Frame", frame)
    
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window



(668, 88)
(668, 503, 3)


-1

In [49]:

camera = cv2.VideoCapture(1)
print camera.isOpened()
camera.release()


True
